In [ ]:
import time
import math
import random
from typing import List, Tuple
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    PURPLE = '\033[95m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'

class Connect4Plus:
    def __init__(self):
        """Initialize the Connect 4 game with enhanced features"""
        self.current_stats = {
            'nodes_explored': 0,
            'pruning_events': 0,
            'depth_stats': defaultdict(int),
            'branching_factors': [],
            'evaluation_history': [],
            'time_taken': 0
        }
        self.transposition_table = {}

        # Game board setup
        self.number_of_rows = 6
        self.number_of_columns = 7
        self.game_board = [[' ' for _ in range(self.number_of_columns)] for _ in range(self.number_of_rows)]
        self.human_piece = '●'
        self.computer_piece = '○'
        self.pieces_needed_to_win = 4

        # Game statistics
        self.human_score = 0
        self.computer_score = 0
        self.move_history = []
        self.pruning_history = []
        self.minimax_stats = []

        # Evaluation weights
        self.weights = {
            'win': 1000000,
            'three_in_row': 100,
            'two_in_row': 10,
            'center_control': 3,
            'threat': 50,
            'block': 40,
            'connection': 5
        }

    def display_game_board(self):
        """Display the current game board state"""
        print(f"\n{Colors.BOLD}=== Connect 4 Plus ==={Colors.ENDC}")
        print(f"Score - Human: {self.human_score} | Computer: {self.computer_score}\n")

        # Column numbers
        print(" " + " ".join(str(i) for i in range(self.number_of_columns)))

        # Board contents
        for row in self.game_board:
            print("|", end="")
            for cell in row:
                if cell == self.human_piece:
                    print(f"{Colors.GREEN}{cell}{Colors.ENDC}", end="|")
                elif cell == self.computer_piece:
                    print(f"{Colors.RED}{cell}{Colors.ENDC}", end="|")
                else:
                    print(" ", end="|")
            print()

        # Bottom border
        print("-" * (self.number_of_columns * 2 + 1))

    def order_moves(self, valid_moves: List[int], maximizing_player: bool) -> List[int]:
        """Order moves for better pruning efficiency"""
        move_scores = []
        piece = self.computer_piece if maximizing_player else self.human_piece
        opp_piece = self.human_piece if maximizing_player else self.computer_piece

        for move in valid_moves:
            row = self.find_next_empty_spot(move)
            if row == -1:
                continue
            score = 0

            # Check for immediate win
            self.game_board[row][move] = piece
            if self.has_player_won(piece):
                score += 10000  # High score for winning move
            self.game_board[row][move] = ' '

            # Check for blocking opponent win
            self.game_board[row][move] = opp_piece
            if self.has_player_won(opp_piece):
                score += 5000  # High score for blocking move
            self.game_board[row][move] = ' '

            # Prefer center columns
            score += (3 - abs(move - self.number_of_columns // 2)) * 10

            # Evaluate potential future threats and position strength
            score += self.evaluate_future_threats(row, move, opp_piece) * 2 # Increased weight for threats
            score += self._evaluate_position_strength(row, move, piece)

            move_scores.append((move, score))

        # Sort moves based on scores, prioritizing higher scores first
        move_scores.sort(key=lambda x: x[1], reverse=True)

        # Return only the sorted move list
        return [move for move, score in move_scores]

    def evaluate_future_threats(self, row: int, col: int, piece: str) -> int:
        """Evaluate potential future threats from opponent using a more efficient approach."""
        return self._evaluate_lines(row, col, piece, threat_weight=5)

    def _evaluate_position_strength(self, row: int, col: int, piece: str) -> int:
        """Evaluate the strength of a position using a more concise approach."""
        return self._evaluate_lines(row, col, piece, connection_weight=10) + (self.number_of_rows - row) * 5

    def _evaluate_lines(self, row: int, col: int, piece: str, threat_weight=0, connection_weight=0) -> int:
        """Helper function to evaluate lines in all directions."""
        total_score = 0
        for dr, dc in [(0, 1), (1, 0), (1, 1), (1, -1)]:  # All directions
            line_score = 0

            # Check both directions along the line
            for d in [-1, 1]:
                count = 0
                for i in range(1, 4):  # Check up to 3 spaces away
                    r, c = row + dr * i * d, col + dc * i * d
                    if 0 <= r < self.number_of_rows and 0 <= c < self.number_of_columns:
                        if self.game_board[r][c] == piece:
                            count += 1
                        else:
                            break  # Stop if the line is broken
                    else:
                        break  # Stop if out of bounds

                if threat_weight:
                    line_score += count * threat_weight
                if connection_weight:
                    line_score += count * connection_weight

            total_score += line_score

        return total_score

    def evaluate_position(self) -> float:
        """Evaluate the current board position"""
        if self.has_player_won(self.computer_piece):
            return self.weights['win']
        if self.has_player_won(self.human_piece):
            return -self.weights['win']

        score = 0

        # Center control preference
        center_col = self.number_of_columns // 2
        center_count = sum(1 for row in range(self.number_of_rows)
                         if self.game_board[row][center_col] == self.computer_piece)
        score += center_count * self.weights['center_control']

        # Evaluate windows of 4
        for row in range(self.number_of_rows):
            for col in range(self.number_of_columns - 3):
                window = [self.game_board[row][col + i] for i in range(4)]
                score += self._evaluate_window(window)

        for row in range(self.number_of_rows - 3):
            for col in range(self.number_of_columns):
                window = [self.game_board[row + i][col] for i in range(4)]
                score += self._evaluate_window(window)

        for row in range(self.number_of_rows - 3):
            for col in range(self.number_of_columns - 3):
                window = [self.game_board[row + i][col + i] for i in range(4)]
                score += self._evaluate_window(window)
                window = [self.game_board[row + 3 - i][col + i] for i in range(4)]
                score += self._evaluate_window(window)

        # Track evaluation history
        self.current_stats['evaluation_history'].append(score)

        return score

    def _evaluate_window(self, window: List[str]) -> int:
        """Evaluate a window of 4 positions"""
        score = 0
        comp_count = window.count(self.computer_piece)
        human_count = window.count(self.human_piece)
        empty_count = window.count(' ')

        # Adjusted weights for evaluation based on game phase
        if self.is_mid_game():
            if comp_count == 4:
                score += self.weights['win']
            elif comp_count == 3 and empty_count == 1:
                score += self.weights['three_in_row'] * 3  # Increased weight for mid-game
            elif comp_count == 2 and empty_count == 2:
                score += self.weights['two_in_row'] * 3  # Increased weight for mid-game

            if human_count == 3 and empty_count == 1:
                score -= self.weights['three_in_row'] * 3  # Increased weight for mid-game
            elif human_count == 2 and empty_count == 2:
                score -= self.weights['two_in_row'] * 3  # Increased weight for mid-game
        else:
            # Early game weights
            if comp_count == 4:
                score += self.weights['win']
            elif comp_count == 3 and empty_count == 1:
                score += self.weights['three_in_row'] * 2  # Early game weight
            elif comp_count == 2 and empty_count == 2:
                score += self.weights['two_in_row'] * 2  # Early game weight

            if human_count == 3 and empty_count == 1:
                score -= self.weights['three_in_row'] * 2  # Early game weight
            elif human_count == 2 and empty_count == 2:
                score -= self.weights['two_in_row'] * 2  # Early game weight

        return score

    def is_mid_game(self) -> bool:
        """Determine if the game is in the mid-game phase"""
        return len(self.move_history) > 10  # Example threshold for mid-game

    def make_move(self, column: int, piece: str) -> bool:
        """Make a move in the specified column"""
        row = self.find_next_empty_spot(column)
        if row != -1:
            self.game_board[row][column] = piece
            return True
        return False

    def find_next_empty_spot(self, column: int) -> int:
        """Find the next empty spot in the column"""
        for row in range(self.number_of_rows - 1, -1, -1):
            if self.game_board[row][column] == ' ':
                return row
        return -1

    def is_valid_move(self, column: int) -> bool:
        """Check if a move is valid"""
        return 0 <= column < self.number_of_columns and self.find_next_empty_spot(column) != -1

    def get_valid_moves(self) -> List[int]:
        """Get all valid moves"""
        return [col for col in range(self.number_of_columns)
                if self.find_next_empty_spot(col) != -1]

    def is_board_full(self) -> bool:
        """Check if the board is full"""
        return all(self.game_board[0][col] != ' ' for col in range(self.number_of_columns))

    def has_player_won(self, piece: str) -> bool:
        """Check if a player has won"""
        # Horizontal check
        for row in range(self.number_of_rows):
            for col in range(self.number_of_columns - 3):
                if all(self.game_board[row][col + i] == piece for i in range(4)):
                    return True

        # Vertical check
        for row in range(self.number_of_rows - 3):
            for col in range(self.number_of_columns):
                if all(self.game_board[row + i][col] == piece for i in range(4)):
                    return True

        # Diagonal checks
        for row in range(self.number_of_rows - 3):
            for col in range(self.number_of_columns - 3):
                # Positive diagonal
                if all(self.game_board[row + i][col + i] == piece for i in range(4)):
                    return True
                # Negative diagonal
                if all(self.game_board[row + 3 - i][col + i] == piece for i in range(4)):
                    return True

        return False

    def minimax(self, depth: int, alpha: float, beta: float, maximizing_player: bool) -> Tuple[int, float]:
        """Minimax algorithm with enhanced alpha-beta pruning"""
        self.current_stats['nodes_explored'] += 1
        self.current_stats['depth_stats'][depth] += 1

        # Terminal checks
        if self.has_player_won(self.computer_piece):
            return None, self.weights['win'] + depth
        if self.has_player_won(self.human_piece):
            return None, -self.weights['win'] - depth
        if self.is_board_full() or depth == 0:
            return None, self.evaluate_position()

        valid_moves = self.get_valid_moves()
        valid_moves = self.order_moves(valid_moves, maximizing_player)
        self.current_stats['branching_factors'].append(len(valid_moves))

        if maximizing_player:
            value = -math.inf
            best_move = valid_moves[0] if valid_moves else None

            for col in valid_moves:
                row = self.find_next_empty_spot(col)
                if row != -1:
                    self.game_board[row][col] = self.computer_piece
                    new_score = self.minimax(depth - 1, alpha, beta, False)[1]
                    self.game_board[row][col] = ' '

                    if new_score > value:  # Update value if better score is found
                        value = new_score
                        best_move = col
                    alpha = max(alpha, value)  # Update alpha

                    if alpha >= beta:
                        self.current_stats['pruning_events'] += len(valid_moves) - valid_moves.index(col) + 1  # Accurate pruning count
                        break

            return best_move, value
        else:
            value = math.inf
            best_move = valid_moves[0] if valid_moves else None

            for col in valid_moves:
                row = self.find_next_empty_spot(col)
                if row != -1:
                    self.game_board[row][col] = self.human_piece
                    new_score = self.minimax(depth - 1, alpha, beta, True)[1]
                    self.game_board[row][col] = ' '

                    if new_score < value:  # Update value if better score is found
                        value = new_score
                        best_move = col
                    beta = min(beta, value)  # Update beta

                    if alpha >= beta:
                        self.current_stats['pruning_events'] += len(valid_moves) - valid_moves.index(col) + 1  # Accurate pruning count
                        break

            return best_move, value

    def computer_move(self) -> None:
        """Make a move for the computer player"""
        start_time = time.time()
        self.current_stats = {
            'nodes_explored': 0,
            'pruning_events': 0,
            'depth_stats': defaultdict(int),
            'branching_factors': [],
            'evaluation_history': [],
            'time_taken': 0
        }

        best_move = None
        best_score = -math.inf
        depth = 1
        time_limit = 5  # Set a time limit for the move in seconds
        time_limit = 1  # Reduced time limit to 1 second

        while True:
            move, score = self.minimax(depth, -math.inf, math.inf, True)
            if time.time() - start_time > time_limit:
                break  # Stop if the time limit is exceeded
            if move is not None:
                best_move = move
                best_score = score
            depth += 1  # Increase depth for the next iteration


        if best_move is not None:
            row = self.find_next_empty_spot(best_move)
            if row != -1:
                self.game_board[row][best_move] = self.computer_piece
                end_time = time.time()

                self.move_history.append({
                    'column': best_move,
                    'time': end_time - start_time,
                    'depth': depth,
                    'nodes': self.current_stats['nodes_explored'],
                    'pruning': self.current_stats['pruning_events'],
                    'avg_branching': sum(self.current_stats['branching_factors']) / len(self.current_stats['branching_factors']) if self.current_stats['branching_factors'] else 0
                })

                # Visualize search space analysis and pruning efficiency
                self.visualize_search_space_analysis()

    def play(self):
        """Main game loop"""
        print("\nWelcome to Connect 4!")

        while True:
            self.display_game_board()

            # Human's turn
            while True:
                try:
                    column = int(input("\nYour turn! Choose column (0-6): "))
                    if self.is_valid_move(column):
                        self.make_move(column, self.human_piece)
                        break
                    else:
                        print("Invalid move. Try again.")
                except ValueError:
                    print("Please enter a number between 0 and 6.")

            self.display_game_board()

            if self.has_player_won(self.human_piece):
                print("Congratulations! You win!")
                self.human_score += 1
                break
            elif self.is_board_full():
                print("It's a tie!")
                break

            # Computer's turn
            print("Computer thinking...")
            self.computer_move()

            if self.has_player_won(self.computer_piece):
                self.display_game_board()
                print("Computer wins!")
                self.computer_score += 1
                break
            elif self.is_board_full():
                self.display_game_board()
                print("It's a tie!")
                break

        # Display final statistics
        print("\n=== Game Statistics ===")
        print(f"Total moves: {len(self.move_history)}")
        print(f"Game duration: {sum(move['time'] for move in self.move_history):.1f} seconds\n")

        print("\nAI Decision Analysis:")
        total_nodes = sum(move['nodes'] for move in self.move_history)
        total_pruning = sum(move['pruning'] for move in self.move_history)
        print(f"Total nodes explored: {total_nodes:,}")
        print(f"Pruning events: {total_pruning:,}")
        print(f"Overall Pruning Efficiency of the Game: {(total_pruning / (total_nodes + total_pruning)) * 100 if total_nodes + total_pruning > 0 else 0:.1f}% (Formula: Nodes Pruned / (Nodes Explored + Nodes Pruned) * 100)")

        self.visualize_performance()

    def visualize_performance(self):
        """Visualize AI performance metrics"""
        if not self.move_history:
            print("No moves to visualize yet.")
            return

        plt.style.use('default')
        fig = plt.figure(figsize=(15, 10))

        # 1. Decision Time Analysis
        ax1 = plt.subplot(221)
        moves = range(1, len(self.move_history) + 1)
        times = [move['time'] for move in self.move_history]

        ax1.plot(moves, times, 'b-', marker='o')
        ax1.set_title('Decision Time per Move')
        ax1.set_xlabel('Move Number')
        ax1.set_ylabel('Time (seconds)')
        ax1.grid(True)

        # Add moving average
        window = 3
        if len(times) >= window:
            moving_avg = [sum(times[i:i+window])/window
                        for i in range(len(times)-window+1)]
            ax1.plot(moves[window-1:], moving_avg, 'r--',
                    label=f'{window}-Move Average')
        ax1.legend()

        # 2. Nodes Explored
        ax2 = plt.subplot(222)
        nodes = [move['nodes'] for move in self.move_history]
        pruning = [move['pruning'] for move in self.move_history]

        ax2.bar(moves, nodes, label='Nodes Explored')
        ax2.bar(moves, pruning, bottom=nodes, label='Nodes Pruned')
        ax2.set_title('Search Space Analysis')
        ax2.set_xlabel('Move Number')
        ax2.set_ylabel('Number of Nodes')
        ax2.legend()

        # 3. Pruning Efficiency
        ax3 = plt.subplot(223)
        efficiency = [(p/(n+p))*100 if n+p > 0 else 0
                     for n, p in zip(nodes, pruning)]

        ax3.plot(moves, efficiency, 'g-', marker='s')
        ax3.set_title('Pruning Efficiency')
        ax3.set_xlabel('Move Number')
        ax3.set_ylabel('Efficiency (%)')
        ax3.grid(True)

        # 4. Pie Chart: Nodes Explored vs. Pruning Events
        ax4 = plt.subplot(224)  # Now ax4 is used for the pie chart
        total_nodes = sum(move['nodes'] for move in self.move_history)
        total_pruning = sum(move['pruning'] for move in self.move_history)

        labels = ['Total Nodes Explored', 'Pruning Events']
        sizes = [total_nodes, total_pruning]
        colors = ['lightblue', 'lightgreen']
        explode = (0.1, 0)  # Explode the first slice (Nodes Explored)

        ax4.pie(sizes, explode=explode, labels=labels, colors=colors,
                autopct='%1.1f%%', shadow=True, startangle=140)
        ax4.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
        ax4.set_title('Search Space Analysis: Nodes vs. Pruning')

        plt.tight_layout()
        plt.show()

        # Print summary statistics
        total_nodes = sum(nodes)
        total_pruning = sum(pruning)
        avg_time = sum(times) / len(times)

        print("\nPerformance Summary:")
        print(f"Total nodes explored: {total_nodes:,}")
        print(f"Total nodes pruned: {total_pruning:,}")
        print(f"Overall Pruning Efficiency of the Game: {(total_pruning / (total_nodes + total_pruning)) * 100 if total_nodes + total_pruning > 0 else 0:.1f}% (Formula: Nodes Pruned / (Nodes Explored + Nodes Pruned) * 100)")
        print(f"Average decision time: {avg_time:.3f} seconds")
        print(f"Total moves: {len(self.move_history)}")

    def visualize_search_space_analysis(self):
        """Visualize search space analysis and pruning efficiency."""
        nodes = self.current_stats['nodes_explored']
        pruning = self.current_stats['pruning_events']
        efficiency = (pruning / (nodes + pruning)) * 100 if nodes + pruning > 0 else 0
        print(f"\nSearch Space: {nodes} explored, {pruning} pruned ({efficiency:.1f}% efficiency)")

    def check_two_in_a_row(self, row: int, col: int, piece: str) -> bool:
        """Check if there are two pieces in a row."""
        return self._check_n_in_a_row(row, col, piece, 2)

    def _check_n_in_a_row(self, row: int, col: int, piece: str, n: int) -> bool:
        """Check for n pieces in a row in any direction (horizontal, vertical, diagonal)."""
        for dr, dc in [(0, 1), (1, 0), (1, 1), (1, -1)]:  # All directions
            count = 0
            for i in range(n):
                r, c = row + dr * i, col + dc * i
                if 0 <= r < self.number_of_rows and 0 <= c < self.number_of_columns and self.game_board[r][c] == piece:
                    count += 1
                else:
                    break
            if count == n:
                return True

            # Check in the opposite direction
            count = 0
            for i in range(n):
                r, c = row - dr * i, col - dc * i
                if 0 <= r < self.number_of_rows and 0 <= c < self.number_of_columns and self.game_board[r][c] == piece:
                    count += 1
                else:
                    break
            if count == n:
                return True

        return False

if __name__ == "__main__":
    game = Connect4Plus()
    game.play()





Welcome to Connect 4!

=== Connect 4 Plus ===
Score - Human: 0 | Computer: 0

 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
---------------

Your turn! Choose column (0-6): 0

=== Connect 4 Plus ===
Score - Human: 0 | Computer: 0

 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|●| | | | | | |
---------------
Computer thinking...

Search Space: 373 explored, 664 pruned (64.0% efficiency)

=== Connect 4 Plus ===
Score - Human: 0 | Computer: 0

 0 1 2 3 4 5 6
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|●| | |○| | | |
---------------
